In [1]:
import platform
print(platform.python_version())

3.12.3


Create and enable to virtualenv, using uv

In [2]:
!uv sync --active

Resolved 138 packages in 0.63ms
Audited 112 packages in 0.92ms


Start the exercise

In [3]:
# Import necessary libraries
import os
from typing import Any, Dict

import google.generativeai as genai
import requests
from dotenv import load_dotenv

print("We are up and running!")

We are up and running!


Ensure that you created a Gemini API Key and added it to the `.env` file before executing the next command

In [4]:
# Load environment variables
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if not api_key or api_key.startswith("ADD YOUR"):
    raise ValueError("GEMINI_API_KEY not found in .env file")

# Configure Gemini
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-3-flash-preview")

print(f"Gemini model loaded successfully: {model.model_name}")

Gemini model loaded successfully: models/gemini-3-flash-preview


In [5]:
PROMPT = "What is the current price of Bitcoin?"
chat = model.start_chat()
response = chat.send_message(PROMPT)
print(response.text)

As of **May 22, 2024**, Bitcoin is currently trading at approximately **$70,150 USD**.

Please note that the cryptocurrency market is extremely volatile, and the price can change significantly within minutes. For the most accurate real-time data, you should check a live tracker such as:

*   [CoinMarketCap](https://coinmarketcap.com/currencies/bitcoin/)
*   [CoinGecko](https://www.coingecko.com/en/coins/bitcoin)
*   [Binance](https://www.binance.com/en/price/bitcoin) or [Coinbase](https://www.coinbase.com/price/bitcoin)


In [6]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "As of **May 22, 2024**, Bitcoin is currently trading at approximately **$70,150 USD**.\n\nPlease note that the cryptocurrency market is extremely volatile, and the price can change significantly within minutes. For the most accurate real-time data, you should check a live tracker such as:\n\n*   [CoinMarketCap](https://coinmarketcap.com/currencies/bitcoin/)\n*   [CoinGecko](https://www.coingecko.com/en/coins/bitcoin)\n*   [Binance](https://www.binance.com/en/price/bitcoin) or [Coinbase](https://www.coinbase.com/price/bitcoin)"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 9,
        "candidates_token_cou

In [7]:
url = f"https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"
response = requests.get(url)
data = response.json()
print(data)

{'symbol': 'BTCUSDT', 'price': '89619.14000000'}


In [8]:

# Define the function
def get_crypto_price(symbol: str) -> Dict[str, Any]:
    """
    Get the current price of a cryptocurrency from Binance API
    """
    url = f"https://api.binance.com/api/v3/ticker/price?symbol={symbol}"
    response = requests.get(url)
    data = response.json()
    return float(data["price"])


In [ ]:
price = get_crypto_price("BTCUSDT")
print(f"BTC Price in USDT: {price}")

BTC Price in USDT: 89619.14


In [11]:
tools = [
    {
        "function_declarations": [
            {
                "name": "get_crypto_price",
                "description": "Get cryptocurrency price in USDT from Binance",
                "parameters": {
                    "type": "object", 
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The cryptocurrency trading pair symbol (e.g., BTCUSDT, ETHUSDT). \
                                            The symbol for Bitcoin is BTCUSDT. \
                                            The symbol for Ethereum is ETHUSDT."
                        }
                    },
                    "required": ["symbol"]
                }
            }
        ]
    }
]

In [12]:
PROMPT = "What is the current price of Bitcoin?"
chat = model.start_chat()
response = chat.send_message(PROMPT, tools=tools)
print(response)


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_crypto_price",
                  "args": {
                    "symbol": "BTCUSDT"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 100,
        "candidates_token_count": 20,
        "total_token_count": 184
      },
      "model_version": "gemini-3-flash-preview"
    }),
)


In [13]:
price = get_crypto_price("BTCUSDT")
# print(price) # enable only to DEBUG because it breaks next steps

In [14]:
final_response = chat.send_message(str(price))
print(final_response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The current price of Bitcoin (BTC) is approximately **$89,645.44**."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 37,
        "candidates_token_count": 21,
        "total_token_count": 119
      },
      "model_version": "gemini-3-flash-preview"
    }),
)


In [15]:
print(final_response.text)

The current price of Bitcoin (BTC) is approximately **$89,645.44**.


----